In [1]:
import sys
sys.path.append('/home/fernando/Proyectos/PÓS-GRADUAÇÂO/TESIS-DOUTORADO-2/PESQUISA/tools/cnn_patient_people/library');
sys.path.append('/home/fernando/Proyectos/PÓS-GRADUAÇÂO/TESIS-DOUTORADO-2/PESQUISA/libraries/WorkingWithFiles/src');

In [ ]:
import PIL
import os
import PatientPeopleLib as mylib
import WorkingWithFiles as rnfunc
from tensorflow.keras.preprocessing.image import load_img

# Variables globales

In [2]:
## Seed for the random variables
seed_number=0;

## Model of network
#model_type = 'efficientnet_b3'
model_type = 'mobilenet_v3';
#model_type = 'inception_v3';
#model_type = 'inception_resnet_v2';
#model_type = 'resnet_v2_50';

times=10;

## Output
output_base_dir = '/mnt/boveda/DOCTORADO2/cnn_patient_people/test_custom';

# If command line

In [3]:
#print('cmd entry:', sys.argv)

for n in range(len(sys.argv)):
    if sys.argv[n]=='--model':
        model_type=sys.argv[n+1];
    if sys.argv[n]=='--times':
        times=int(sys.argv[n+1]);

print('model_type:',model_type)
print('times:',times)

cmd entry: ['/usr/lib/python3/dist-packages/ipykernel_launcher.py', '-f', '/home/fernando/.local/share/jupyter/runtime/kernel-9ff4a45b-e0b2-45f2-b105-7a0ffe931e2a.json']
model_type: mobilenet_v3


# Classifier

In [4]:
Clf=mylib.PatientClassifier(model_type);

# Create directory

In [5]:
#output_dir = os.path.join(output_base_dir,'holdout');
output_dir = os.path.join(output_base_dir,'delay_'+model_type);
try: 
    os.mkdir(output_base_dir) 
except: 
    pass

try: 
    os.mkdir(output_dir) 
except: 
    pass

# Time

In [6]:
from pytictoc import TicToc
t = TicToc() 

# Testing people

In [7]:

###############
basedir='/mnt/boveda/DATASETs/PATIENT-IMAGES/patient_people/test/people';
total_list=rnfunc.get_all_files_in_dir_list([basedir],formats_search=['.png']);

N0=0;
L0=len(total_list);

pil_image=[];
for n in range(L0):
    pil_image.append(load_img(total_list[n]));

t.tic();
for m in range(times):
    for n in range(L0):
        res=Clf.is_pil_patient(pil_image[n]);
        N0=N0+(res==0);
t0=t.tocvalue();

print('Acc0:',N0*1.0/(L0*times));
print('t0:',t0);

Acc0: 0.9923076923076923
t0: 5.876880223000626


# Testing patient

In [8]:
basedir='/mnt/boveda/DATASETs/PATIENT-IMAGES/patient_people/test/patient';
total_list=rnfunc.get_all_files_in_dir_list([basedir],formats_search=['.png']);

N1=0;
L1=len(total_list);

pil_image=[];
for n in range(L1):
    pil_image.append(load_img(total_list[n]));

t.tic();
for m in range(times):
    for n in range(L1):
        res=Clf.is_pil_patient(pil_image[n]);
        N1=N1+(res==1);
t1=t.tocvalue();

print('Acc1:',N1*1.0/(L1*times));
print('t1:',t1);

Acc1: 0.916083916083916
t1: 6.2609743180000805


# Resultados finais

In [9]:
acc=(N0+N1)*1.0/((L0+L1)*times);
delayms=(t1+t0)*1000.0/((L0+L1)*times);

print('Acc:',acc);
print('Delay',delayms,'ms')

Acc: 0.9523809523809523
Delay 44.461005644691234 ms


In [10]:
fpath=os.path.join(output_dir,"times"+str(times)+"_acc_delayms.m")

fid = open(fpath, 'w')
print('delayms={}'.format(delayms),';', file = fid);
print('acc={}'.format(acc),';', file = fid);
fid.close()